In [ ]:
from keras import Sequential
from sklearn.model_selection import train_test_split
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adam
from keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping
from keras.layers import Flatten, Dense, Dropout, BatchNormalization, UpSampling2D
from keras.utils import to_categorical
from keras.datasets import cifar10
import pandas as pd
import pickle 

def resnet_accuracies(froze=False, LR_scheduler='Plateau', augmentation=True, batch_size=1000, epochs=30, learn_rate=.001):
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train = preprocess_input(x_train)
    x_test = preprocess_input(x_test)
    
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3)

    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    y_test = to_categorical(y_test)

    train_generator = ImageDataGenerator(
        rotation_range=2,
        horizontal_flip=True,
        zoom_range=.1)

    val_generator = ImageDataGenerator(
        rotation_range=2,
        horizontal_flip=True,
        zoom_range=.1)

    test_generator = ImageDataGenerator(
        rotation_range=2,
        horizontal_flip=True,
        zoom_range=.1)
    train_generator.fit(x_train)
    val_generator.fit(x_val)
    test_generator.fit(x_test) 
    
    # setting learning rate schedulers
    def scheduler(epoch):
        if epoch < 5:
            return learn_rate 
        if epoch < 20:
            return learn_rate /10
        return learn_rate/100
    if LR_scheduler == 'Custom' :
        callbacks = [LearningRateScheduler(scheduler)]
    elif LR_scheduler == 'Plateau' :
        callbacks = [ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=.01,
        patience=3,
        min_lr=1e-5)]
    else :
        callbacks = [LearningRateScheduler(lambda x: learn_rate)]
    base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(128, 128, 3),
                          classes=y_train.shape[1])

    callbacks.append(EarlyStopping(monitor='val_loss', patience=5, min_delta=0.01))
    model = Sequential()
    model.add(UpSampling2D(size=(2,2))) 
    model.add(UpSampling2D(size=(2,2))) 
    model.add(base_model)
    model.add(Flatten())
    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(.5))
    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    model.layers[3].trainable = not froze

    adam = Adam(lr=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    if augmentation:
        history = model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size),
                  epochs=epochs, steps_per_epoch=x_train.shape[0] // batch_size,
                  validation_data=val_generator.flow(x_val, y_val, batch_size=batch_size), validation_steps=10,
                  callbacks=callbacks, verbose=1)
    else:
        history = model.fit(x_train, y_train, batch_size=batch_size,
                  epochs=epochs, steps_per_epoch=x_train.shape[0] // batch_size,
                  validation_data=(x_val, y_val), validation_steps=10,
                  callbacks=callbacks, verbose=1)
    return {**history.history, 'test_accuracy': (model.predict(x_test).argmax(axis=1) == y_test.argmax(axis=1)).mean()}



In [ ]:
gpu_info = !nvidia-smi
'\n'.join(gpu_info)

'Sun Apr 18 13:11:14 2021       \n+-----------------------------------------------------------------------------+\n| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |\n|-------------------------------+----------------------+----------------------+\n| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |\n| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |\n|                               |                      |               MIG M. |\n|===============================+======================+======================|\n|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |\n| N/A   38C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |\n|                               |                      |                  N/A |\n+-------------------------------+----------------------+----------------------+\n                                                                           

In [3]:

experiments = [{'name': 'Base model',
                'kwargs': {}},
               {'name': 'Frozen',
                'kwargs': {'froze': True}},
               {'name': 'No scheduler',
                'kwargs': {'LR_scheduler': None}},
               {'name': 'Custom scheduler',
                'kwargs': {'LR_scheduler': 'Custom'}},
               {'name': 'No augmentation',
                'kwargs': {'augmentation': False}},
              ]
for i in range(2):
    assessment = pd.DataFrame([{**resnet_accuracies(**exp['kwargs']),
                               'name': exp['name']} for exp in experiments])
    assessment.to_pickle(f'assessment_{i}.pkl')

Epoch 1/30
350/350 [==============================] - 69s 181ms/step - loss: 1.2421 - accuracy: 0.5941 - val_loss: 1.1243 - val_accuracy: 0.6600
Epoch 2/30
350/350 [==============================] - 62s 178ms/step - loss: 0.6250 - accuracy: 0.8012 - val_loss: 0.5964 - val_accuracy: 0.8000
Epoch 3/30
350/350 [==============================] - 62s 178ms/step - loss: 0.5293 - accuracy: 0.8244 - val_loss: 0.9413 - val_accuracy: 0.7180
Epoch 4/30
350/350 [==============================] - 62s 177ms/step - loss: 0.4665 - accuracy: 0.8471 - val_loss: 0.6496 - val_accuracy: 0.7840
Epoch 5/30
350/350 [==============================] - 62s 177ms/step - loss: 0.4131 - accuracy: 0.8657 - val_loss: 0.9394 - val_accuracy: 0.7220
Epoch 6/30
350/350 [==============================] - 62s 177ms/step - loss: 0.3405 - accuracy: 0.8924 - val_loss: 0.3396 - val_accuracy: 0.8870
Epoch 7/30
350/350 [==============================] - 62s 177ms/step - loss: 0.2943 - accuracy: 0.9032 - val_loss: 0.3366 - val_ac